In [20]:
''' This code generates an autoregressive time series with added normal noise given its order and initial values 
    and stores the data in the Data.xlsx file. 
    specify the row to store the data in variable r
'''
# set row in Data.xlsx file
r = 1
# set standard deviation
sigma = 0.1
# set path of file
path = "Data.xlsx"

import numpy as np
import pandas as pd
import openpyxl
import copy
# Get which order the user wants for the autoregressive model
order = int(input("Order for Autoregressive model"))

# create an array of shape(order, ) and input coefficients in it
coeff = np.zeros((order, ))
for idx in range(order) :
    coeff[idx] = float(input("coefficient : " + str(idx + 1)))
coeff.reshape((-1, 1))
const = float(input("coefficiet : "+ str(order+1)))
# print(coeff)

# get the initial values of the time series
initial = np.zeros((order, ))
for idx in range(order):
    initial[idx] = float(input("initial value at time " + str(idx + 1) + " : "))
# print (initial)

# get number of time steps
len = int(input("Length of time series to be generated : "))

# write initial data in the excel file
wb = openpyxl.load_workbook(path)
ws = wb.active

for idx in range(order):
    ws.cell(row=r, column=idx+1).value = initial[idx]

# generate the autoregressive time series of the given order
# latest stores the latest values of the times series
latest = copy.deepcopy(initial)

for idx in range(len):
    # multiply the matrices
    val = np.matmul(latest, coeff) + const + np.random.normal(0, sigma)
    np.roll(latest, -1)
    latest[order-1] = val
    ws.cell(row=r, column=order+idx+1).value = val

wb.save(path)

In [21]:
''' This code is used for retreiving the parameters of an AR process
'''
def prediction(coefficients, laggedTargets):
    predictionValue = coefficients[0]
    for i in range(laggedTargets.size):
        predictionValue = coefficients[i + 1] * laggedTargets[i]
    return predictionValue

def findCoefficients(data, k):
    size = data.size
    data_old = np.array([]).reshape(size - k, 0)
    data_new = data[k:size]
    k -= 1
    size -= 1
    while(k >= 0):
        data_old = np.c_[data_old, data[k:size]]
        k -= 1
        size -= 1
    
    data_old = np.c_[data_old, np.ones(size - k)]
    Y = data_new.reshape(size - k, 1)
    A = data_old
    print("------A matrix------")
    print(A)
    print("------Y matrix------")
    print(Y)
    theta = np.matmul(np.matmul(np.linalg.pinv(np.matmul(A.T, A)), A.T), Y)
    # theta = np.dot(np.linalg.pinv(A), Y)
    print("------Theta matrix------")
    print(theta)

    coefficients = theta.reshape(data.size - size)
    return coefficients

In [23]:
''' This code is used to read AR data from Data.xlsx file and predict the coefficients
'''
# specify row to read from
r = 1

wb = openpyxl.load_workbook(path)
wa = wb.active
tot_cols = wa.max_column

# try printing
y = np.zeros(tot_cols)
for idx in range(1, tot_cols+1):
    y[idx-1] = wa.cell(row = r, column = idx).value

findCoefficients(y, order)
print("-----original Theta matrix-----")
print(coeff)
print(const)

------A matrix------
[[ 1.          1.        ]
 [ 3.06333514  1.        ]
 [ 4.96948034  1.        ]
 [ 6.90318363  1.        ]
 [ 8.9213772   1.        ]
 [10.87168641  1.        ]
 [12.79104239  1.        ]
 [14.80832674  1.        ]
 [16.62613124  1.        ]
 [18.73540607  1.        ]]
------Y matrix------
[[ 3.06333514]
 [ 4.96948034]
 [ 6.90318363]
 [ 8.9213772 ]
 [10.87168641]
 [12.79104239]
 [14.80832674]
 [16.62613124]
 [18.73540607]
 [20.8086202 ]]
------Theta matrix------
[[1.00268937]
 [1.95432068]]
-----original Theta matrix-----
[1.]
2.0
